## Imports

In [1]:
import pandas as pd
import altair as alt
import tabula
import geopandas as gpd
import folium
from branca.colormap import LinearColormap
import os

import folium
import random

## Data read-in

In [2]:
# Read in data and filter to desired DF
dfs = tabula.read_pdf('2023-news-release-july-1st-preliminary-taxable-values.pdf',pages=2)
df = dfs[0]

gdf = gpd.read_file('Municipal_Boundary.geojson')

In [3]:
# Rename columns
df = df.rename(columns={'Unnamed: 0':'TAXING AUTHORITY',
                                '2022':'2022 PRELIMINARY TAXABLE VALUE',
                                '2023':'2023 TAXABLE VALUE BEFORE NEW CONSTRUCTION',
                               'Unnamed: 1':'PERCENT CHANGE',
                               'Unnamed: 2':'NEW CONSTRUCTION',
                               '2023.1':'2023 PRELIMINARY TAXABLE VALUE',
                               'Unnamed: 3':'PERCENT CHANGE FROM 2022',
                               'Unnamed: 4':'NET VALUE CHANGE'})

In [4]:
# Remove bad rows
df = df.drop(index=[0,1,2,40])
# Reset index
df = df.reset_index()
# Drop old index
df = df.drop(columns='index')

In [5]:
df2 = df[['2023 PRELIMINARY TAXABLE VALUE','PERCENT CHANGE FROM 2022']]

In [6]:
replace_list = ['MIAMI','MIAMI','MIAMI BEACH','NORMANDY SHORES','CORAL GABLES','HIALEAH','MIAMI SPRINGS',
               'NORTH MIAMI','NORTH MIAMI BEACH','OPA-LOCKA','SOUTH MIAMI','HOMESTEAD',
               'MIAMI SHORES','BAL HARBOUR','BAL HARBOUR ISLANDS','SURFSIDE','WEST MIAMI',
               'FLORIDA CITY','BISCAYNE PARK','EL PORTAL','GOLDEN BEACH','PINECREST',
               'INDIAN CREEK','MEDLEY','NORTH BAY VILLAGE','KEY BISCAYNE','SWEETWATER',
               'VIRGINIA GARDENS','HIALEAH GARDENS','AVENTURA','UNINCORPORATED MIAMI-DADE',
               'SUNNY ISLES BEACH','MIAMI LAKES','PALMETTO BAY','MIAMI GARDENS','DORAL',
                'CUTLER BAY','COUNTY-WIDE','FIRE AND RESCUE','LIBRARY','SCHOOL BOARD',
               'WATER MANAGEMENT','INLAND NAV DISTRICT','CHILDRENS TRUST']

In [7]:
df2['TAXING AUTHORITY'] = replace_list

/var/folders/rq/0_j_rksd7psgdycg9c1qwf6m0000gp/T/ipykernel_44159/206029735.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['TAXING AUTHORITY'] = replace_list


In [8]:
final_df = pd.merge(gdf, df2, left_on='NAME', right_on='TAXING AUTHORITY',how='left')

In [9]:
final_df = final_df[['NAME','geometry','2023 PRELIMINARY TAXABLE VALUE','PERCENT CHANGE FROM 2022','TAXING AUTHORITY']]

In [10]:
final_df['2023 PRELIMINARY TAXABLE VALUE'] = '$' + final_df['2023 PRELIMINARY TAXABLE VALUE'] 

In [34]:
final_df.columns

Index(['NAME', 'geometry', '2023 PRELIMINARY TAXABLE VALUE',
       'PERCENT CHANGE FROM 2022', 'TAXING AUTHORITY'],
      dtype='object')

In [43]:
new_line = '<br>'
strong_start = '<strong>'
strong_end = '</strong>'

In [45]:
m = folium.Map(location=[25.5516,-80.6327,],zoom_start=9)

# Define a function to generate a random color
def generate_random_color():
    color = '#{:06x}'.format(random.randint(0, 256**3))
    return color

# Iterate over each row in the GeoDataFrame
for index, row in final_df.iterrows():
    tooltip_text = row[['NAME','2023 PRELIMINARY TAXABLE VALUE','PERCENT CHANGE FROM 2022']]
    color = generate_random_color()
    folium.GeoJson(
        row['geometry'],
        name=tooltip_text,
        style_function=lambda x, fill_color=color: {
            'fillColor': fill_color,
            'fillOpacity': 0.8,
            'weight': 2,
            'color': 'black'
        },
        tooltip=f"{strong_start}Taxing Authority:{strong_end} {row['NAME']}{new_line}{strong_start}Preliminary Taxable Value: {strong_end}{row['2023 PRELIMINARY TAXABLE VALUE']}{new_line}{strong_start} Percent Change From 2022: {strong_end}{row['PERCENT CHANGE FROM 2022']}"
    ).add_to(m)
    
m.save('index.html')
m

In [47]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_Property_Appraiser_Certifies_Taxable_Values


In [51]:
df_for_chart = final_df[['2023 PRELIMINARY TAXABLE VALUE','PERCENT CHANGE FROM 2022','TAXING AUTHORITY']]

In [54]:
df_for_chart = df_for_chart.drop_duplicates()
df_for_chart = df_for_chart.drop(index=[9,62])

In [58]:
df_for_chart.to_csv('MD_new_taxes_for_chart.csv')